# Overview:

* 5B tickets (VEHICLES ADJUDICATED OFF-SITE) 

*  5A tickets (VEHICLES ADJUDICATED ON PARCEL) - are counted from: aDataManagerTicketExport

* NUMBER OF VEHICLES REMOVED ACCORDING TO SMART SHEETS = Smart Sheets Total Vehicle Counts

* TOTAL VEHICLES REMOVED ACCORDING TO RTR TICKETS = 5A + 5B Tickets 

* Matches Removal Ticket Counts - Compares Smart Sheets Vehicle removal counts to total RTR ticket counts(5A + 5B tickets)

* TOTAL RTR TICKETS MINUS SMARTSHEET COUNTS = Total Smart sheet vehicle counts - Total Vehicles Removed According to RTR Tickets (5A + 5B tickets)

## Outputs:
* Vehicle Counts Central Division-Audit


## Downloads needed:
* 2 Smartsheets both Central and South or BOP and ANVIL
* RTR Data - aDataManagerTicketExport


# Libraries

In [1]:
import pandas as pd
import glob
import numpy as np
import os
import smartsheet
import datetime

# Variables

In [2]:
csv = '.csv'
excel = '.xlsx'

In [3]:
while True:
    user = input('What project are we Auditing?: Central or Northern ').lower()
    try:

        #central Set up
        if user == 'central':
            print('Central Audit Started')
            # Counties:
            county= ['ALPINE', 'EL DORADO', 'LAKE', 'NEVADA', 'PLACER', 'TUOLUMNE', 'MENDOCINO', 'TULARE', 'KERN']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'NORTHERN DIVISION'

            # smartsheets Shet IDs
            sheet_id1 = '1844440261257092' # central
            sheet_id2= '6292262490531716' # south central

            # Central's API
            API1 = smartsheet.Smartsheet('6jJFxnAfFsDErU0BSr1Ae4D6cHq0zenczVR12') # API central

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("H8mqAo60J2zGHSrJEwmlMoY88A9yjgCo6RnSI") # API south central
            API2.errors_as_exceptions(True)
            break

        #northern Set up
        elif user == 'northern':
            print('Northern Audit Started')
            # Counties:
            county = ['LASSEN', 'PLUMAS', 'SISKIYOU', 'TRINITY', 'SHASTA', 'TEHAMA']

            # Division. * northern goes here becasue we want everything not in northern Division
            division = 'CENTRAL DIVISION'

            # smartsheets Shet IDs
            sheet_id1 = '3589680131270532' # BOP
            sheet_id2= '5367491607193476' # ANVIL

            # Central's API
            API1 = smartsheet.Smartsheet('6w7vusWNdvdm9TJmjImZfbh0SPQgi3HJaekNb') # API BOP

            API1.errors_as_exceptions(True)

            # Central's API
            API2 = smartsheet.Smartsheet("MQQmCjvvFWLKBoX9K7A33rHSoLnuj1MGVJuVK") # API ANVIL
            API2.errors_as_exceptions(True)
            break
    except ValueError:
        print("\ninvaild input Only 'Central or Northern' allowed")

What project are we Auditing?: Central or Northern northern
Northern Audit Started


# Function to set up Smartsheet into a readable Database

In [4]:
def smartsheet_loadup(sheet):
    columns = [col.title for col in sheet.columns]
    rows = []
    for row in sheet.rows:
        cells = []
        for cell in row.cells:
            cells.append(cell.value)
        rows.append(cells)
    df = pd.DataFrame(rows, columns=columns)
    return df

Area1_SS = smartsheet_loadup(API1.Sheets.get_sheet(sheet_id1))
Area2_SS = smartsheet_loadup(API2.Sheets.get_sheet(sheet_id2))

df_both = pd.concat([Area1_SS,Area2_SS])

#  Functions

In [5]:
def compare(df, comp1=None, comp2=None, comp3=None):
    """Compares two columns"""
    if df[comp1] == df[comp2]:
        return "Match"
    else:
        return "No Match"

### Get ride of setting with copy warning

In [6]:
smartsheets = df_both.copy()

# RTR for vehicle removal

## 5A/5B Tickets -this is the normal RTR unit rate tickets from aDataManagerTicketExport

In [7]:
# this is the normal RTR unit rate tickets from aDataManagerTicketExport
path2 = r"C:\Users\jacque.trahan\Downloads\aDataManagerTicketExport*"
load2 = glob.glob(path2 + excel)

df2 = pd.read_excel(max(load2, key=os.path.getctime), parse_dates=True, na_values=None)

#### Format RTR data

In [8]:
Replace_APNs = {'167-310-2500':'167-310-25-00','028-370-004 21285': '028-370-004_21285',
                '028-370-004 21275':'028-370-004_21275'}

In [9]:
RTR = df2.copy()
RTR = RTR[['Zone Name', 'Service Code', 'Unit Count', 'Is Void', 'GIS Zone1','GIS Zone2','Ticket Number']]
RTR = RTR.copy()
RTR['Zone Name'] = RTR['Zone Name'].str.replace("-","")
RTR['Zone Name'] = RTR['Zone Name'].str[:3] + "-" + RTR['Zone Name'].str[3:6] + "-" + RTR['Zone Name'].str[6:]
RTR['Zone Name'] = RTR['Zone Name'].str.replace('167-310-2500', '167-310-25-00')
RTR['Service Code'] = RTR['Service Code'].str.strip()



#### Format RTR service code data

In [10]:
RTR_Vecs = RTR.copy()
RTR_Vecs = RTR_Vecs[(RTR_Vecs['Is Void'] == False) & (RTR_Vecs['Service Code'].isin(['5A','5B']))]
RTR_Vecs = RTR_Vecs[~RTR_Vecs['GIS Zone1'].isin([division])]

RTR_Vecs = RTR_Vecs[['Zone Name', 'Unit Count', 'Service Code', 'Ticket Number', 'GIS Zone1','GIS Zone2','Is Void']]

# Just 5A Tickets

In [11]:
RTR_5a = RTR_Vecs[RTR_Vecs['Service Code'].isin(['5A'])]
#sum up unit counts
RTR_5a_sum = RTR_5a.groupby(['Zone Name'])[['Unit Count']].sum()

#change Column header
RTR_5a_sum.columns = ["5A Totals(VEHICLES ADJUDICATED ON PARCEL)"]
#reset index
RTR_5a_sum.reset_index(inplace=True)

# Just 5B Tickets

In [12]:
RTR_5b = RTR_Vecs[RTR_Vecs['Service Code'].isin(['5B'])]

#sum up unit counts
RTR_5b_sum = RTR_5b.groupby(['Zone Name'])[['Unit Count']].sum()

# Change column header
RTR_5b_sum.columns = ['5B Totals(VEHICLES ADJUDICATED OFF-SITE)']
#reset index
RTR_5b_sum.reset_index(inplace=True)

# Merge 5A and 5B columns

In [13]:
RTR_Nums = RTR_5a_sum.merge(RTR_5b_sum,
                           how='outer',
                           left_on='Zone Name',
                           right_on='Zone Name')

### Merge smartsheets and rtr total vehicles


In [14]:
smartsheets.columns = map(str.upper, smartsheets.columns)

In [15]:
RTR_vec_final = smartsheets.merge(RTR_Nums,
                             left_on='APN_ROW SEGMENT',
                             right_on='Zone Name',
                             how='left')

In [16]:
RTR_vec_final.rename(columns={'NUMBER OF VEHICLES REMOVED': 'NUMBER OF VEHICLES REMOVED ACCORDING TO SMART SHEETS'},
                     inplace=True)

In [17]:
RTR_vec_final =RTR_vec_final[['APN_ROW SEGMENT','STREET #','STREET NAME','STRUCTURAL STATUS','COUNTY', 'CITY',
                           'TOTAL NUMBER OF VEHICLES','5A Totals(VEHICLES ADJUDICATED ON PARCEL)',
                              '5B Totals(VEHICLES ADJUDICATED OFF-SITE)',
                              'NUMBER OF VEHICLES REMOVED ACCORDING TO SMART SHEETS']]

In [18]:
RTR_vec_final = RTR_vec_final.copy()
RTR_vec_final.fillna(0, inplace=True)

In [19]:
RTR_vec_final['Total Vehicles Removed According to RTR Tickets'] = (RTR_vec_final['5A Totals(VEHICLES ADJUDICATED ON PARCEL)'] + RTR_vec_final['5B Totals(VEHICLES ADJUDICATED OFF-SITE)'])

In [20]:
RTR_vec_final['Total RTR tickets minus Smartsheet Counts'] = RTR_vec_final['TOTAL NUMBER OF VEHICLES'] - RTR_vec_final['Total Vehicles Removed According to RTR Tickets']

In [21]:
RTR_vec_final.columns = map(str.upper, RTR_vec_final.columns)

In [22]:
RTR_vec_final.insert(RTR_vec_final.columns.get_loc('TOTAL VEHICLES REMOVED ACCORDING TO RTR TICKETS')+1, "SMART SHEETS COMPARED TO RTR",
                    RTR_vec_final.apply(compare, comp1='TOTAL VEHICLES REMOVED ACCORDING TO RTR TICKETS',
                                        comp2='NUMBER OF VEHICLES REMOVED ACCORDING TO SMART SHEETS', axis=1))

# Sum the total cars in Smart sheets and total RTR tickets made
* put the sum at the bottom of each row. 

In [23]:
# smart sheet make its own data frame then change colum names to make 
RTR_vec_final.loc['TOTALs'] = RTR_vec_final.sum(numeric_only=True, axis=0).fillna(0)


In [24]:
RTR_vec_final.set_index('APN_ROW SEGMENT', inplace=True)
RTR_vec_final.to_excel('Vehicle Counts Central Division-Audit.xlsx')

### Check Contractor Sheets vs RTR data
### No longer needed

In [25]:
# contractor = pd.read_excel(r'C:\Users\jacque.trahan\Downloads\Vehicles of 2021 CA Fires ADMS Report - RCI.xlsx',
#                            sheet_name='Sub UR Ticket Export')

In [26]:
# contractor = contractor.copy()
# contractor = contractor[['Zone','Quan.','Ticket No.', 'Service Code', 'Collection Addr No', 'Collection Addr ST']]
# contractor['Service Code'] = contractor['Service Code'].str.strip()
# contractor = contractor[(contractor['Service Code'].isin(['5A','5B']))]

# contractor['Zone'] = contractor['Zone'].str.strip()
# contractor['Zone'] = contractor['Zone'].str[:3] + "-" + contractor['Zone'].str[3:6] + "-" + contractor['Zone'].str[6:]

# Sum up contractor 5A

In [27]:
# contractor_5A = contractor.copy()
# contractor_5A = contractor_5A[contractor_5A['Service Code'].isin(['5A'])]
# #sum up unit counts
# contractor_5A_sum = contractor_5A.groupby(['Zone'])[['Quan.']].sum()

# #change Column header
# contractor_5A_sum.columns = ["5A Totals"]
# #reset index
# contractor_5A_sum.reset_index(inplace=True)

# Sum up contractor 5B

In [28]:
# contractor_5B = contractor.copy()

# contractor_5B = contractor.copy()
# contractor_5B = contractor_5B[contractor_5B['Service Code'].isin(['5B'])]
# #sum up unit counts
# contractor_5B_sum = contractor_5B.groupby(['Zone'])[['Quan.']].sum()

# #change Column header
# contractor_5B_sum.columns = ["5B Totals"]
# #reset index
# contractor_5B_sum.reset_index(inplace=True)

In [29]:
# # merge contractor 5A and 5B totals

# contracot_final = contractor_5A_sum.merge(contractor_5B_sum,
#                                          left_on='Zone',
#                                          right_on='Zone',
#                                          how='outer')

In [30]:
# contracot_final = contracot_final.add_suffix("_Contractor Provided Tickets")
# # RTR_vec_final.reset_index(inplace=True)

In [31]:
# vec_final_check = RTR_vec_final.merge(contracot_final,
#                         left_on='APN_ROW SEGMENT',
#                          right_on='Zone_Contractor Provided Tickets',
#                          how='outer')

# vec_final_check.set_index('APN_ROW SEGMENT', inplace=True)
# vec_final_check.to_excel('Tests RTR and Contractor.xlsx')